# Amazon Sales TOP 10

Você é um cientista de dados da Amazon e recebeu a tarefa de investigar os produtos vendidos no site. Seu objetivo é, a partir dos dados, criar uma tabela com os 10 produtos mais caros, com os valores convertidos para BRL.
* [Base Completa de Dados](https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset)

A planilha possui a seguinte estrutura:

|Coluna | Nome da Coluna | Descrição |
|--|--|--|
|A|product_id|ID do produto|
|B|product_name|Nome do produto|
|C|category|Categoria do produto|
|D|actual_price|Valor do produto com desconto (em rúpias indianas)|
|E|product_price|Valor do produto com valor cheio (em rúpias indianas)|
|F|rating|Valor de 0 a 5 avaliando o produto|
|G|rating_count|Quantidade de pessoas que avaliaram o produto|
|H|about_product|Descrição do produto|
|I|product_link|Link do produto na loja da Amazon|

## (E)xtract

In [1]:
import pandas as pd

df = pd.read_csv('dados_amazon.csv')
df.head(3)

,product_id,product_name,category,actual_price,product_price,rating,rating_count,about_product,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,"₹399,00","₹1.099,00","4,5",24269,High Compatibility : Compatible With iPhone 12...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"₹199,00","₹349,00",4,43994,"Compatible with all Type C enabled devices, be...",https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,"₹199,00","₹1.899,00","4,52",7928,【 Fast Charger& Data Sync】-With built-in safet...,https://www.amazon.in/Sounce-iPhone-Charging-C...


Das informações acima, apenas a identificação do produto e seu valor são relevantes para a análise em questão.

In [2]:
top_prices = df[['product_id','product_name','actual_price']]
top_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    1465 non-null   object
 1   product_name  1465 non-null   object
 2   actual_price  1465 non-null   object
dtypes: object(3)
memory usage: 34.5+ KB


## (T)ransform

### Conversão para tipo numérico


As colunas *product_id*, *product_name* e *actual_price* contém 1465 itens não-nulos que pertencem a classe *object*.

A coluna *actual_price* deveria conter valores monetários (numéricos). Contudo, para converter a coluna, é preciso limpar o caractere ₹ (rúpia indiana), além de formatar os pontos e vírgulas.

In [ ]:
# Troca ₹ por nada/. por nada/, por .
top_prices['actual_price'] = top_prices['actual_price'].str.replace('₹', '').str.replace('.', '').str.replace(',', '.')

In [4]:
# Converte a colunas para o tipo numérico
top_prices['actual_price'] = pd.to_numeric(top_prices['actual_price'])

top_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    1465 non-null   object 
 1   product_name  1465 non-null   object 
 2   actual_price  1465 non-null   float64
dtypes: float64(1), object(2)
memory usage: 34.5+ KB


<ipython-input-4-6c8363df5a96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_prices['actual_price'] = pd.to_numeric(top_prices['actual_price'])


### Conversão de INR (₹) para BRL (R$)

Agora os valores na coluna *actual_price* são numéricos, porém ainda estão cotados em rúpias indianas. É preciso, portanto, convertê-los para reais brasileiros.

In [5]:
!pip install requests
import requests

In [6]:
api_key = 'sua-chave-aqui'
base_url = 'https://openexchangerates.org/api/latest.json'

params = {
    'app_id': api_key,
}

response = requests.get(base_url, params=params)

# Verificando se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    data = response.json()
    USD_to_BRL = data['rates']['BRL']
    USD_to_INR = data['rates']['INR']
    print(f"A taxa de câmbio de USD para BRL é: {USD_to_BRL}. \nA taxa de câmbio de USD para INR é: {USD_to_INR}")
else:
    print(f"Erro ao obter a taxa de câmbio. Código de status: {response.status_code}")

A taxa de câmbio de USD para BRL é: 4.9697. 
A taxa de câmbio de USD para INR é: 83.1235


In [7]:
INR_to_BRL = USD_to_INR/USD_to_BRL

In [8]:
top_prices['actual_price_BRL'] = top_prices['actual_price'] / INR_to_BRL
top_prices.head()

<ipython-input-8-32ddae73792b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_prices['actual_price_BRL'] = top_prices['actual_price'] / INR_to_BRL


,product_id,product_name,actual_price,actual_price_BRL
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,23.854990
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,199.0,11.897602
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,199.0,11.897602
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,329.0,19.669904
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,154.0,9.207189


## (L)oad

Com os dados em mãos, resta filtrar os 10 itens vendidos mais caros e exportar as descobertas.

In [9]:
top_prices = top_prices.nlargest(10, 'actual_price')
top_prices

,product_id,product_name,actual_price,actual_price_BRL
249,B09WN3SRC7,Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...,77990.0,4662.783725
325,B095JPKPH3,OnePlus 163.8 cm (65 inches) U Series 4K LED S...,61999.0,3706.730712
255,B0BC8BQ432,VU 164 cm (65 inches) The GloLED Series 4K Sma...,54989.0,3287.624237
192,B0B15GSPQW,Samsung 138 cm (55 inches) Crystal 4K Neo Seri...,47990.0,2869.175420
283,B0B3XXSB1K,LG 139 cm (55 inches) 4K Ultra HD Smart LED TV...,47990.0,2869.175420
332,B09RWQ7YR6,MI 138.8 cm (55 inches) 5X Series 4K Ultra HD ...,46999.0,2809.926559
276,B092BL5DCX,Samsung 138 cm (55 inches) Crystal 4K Series U...,45999.0,2750.139615
533,B0B5V47VK4,"OnePlus 10T 5G (Moonstone Black, 8GB RAM, 128G...",44999.0,2690.352672
216,B095JQVC7N,OnePlus 138.7 cm (55 inches) U Series 4K LED S...,42999.0,2570.778785
1354,B09NS5TKPN,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,42989.0,2570.180915


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(top_prices, *['actual_price'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(top_prices, *['actual_price_BRL'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(top_prices, *['actual_price'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(top_prices, *['actual_price_BRL'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(top_prices, *[[['actual_price', 'actual_price_BRL']]], **{})
chart

## ---
Feito por [cla-isse](https://github.com/cla-isse) 💜